# Test experimental model

This notebook tests the model used for experimental cells runs correctly on a simple step stimulus.

In [ ]:
import json
import sys
import os
import time
import neuron
from pathlib import Path
import time

import matplotlib.pyplot as plt

import bluepyopt as bpopt
import bluepyopt.ephys as ephys

import numpy as np

import shutil
from pprint import pprint

import multimodalfitting as mf

import neuroplotlib as nplt

%matplotlib notebook

In [ ]:
model_name = "experimental" # "hay", "hay_ais", "hay_ais_hillock"
probe_type = "planar"
base_folder = Path("../..")
cell_models_folder = base_folder / "cell_models"

model_folder = cell_models_folder / f"{model_name}_model"
data_folder = base_folder / "data_210916" / "experimental" / "cell1_210301"

In [ ]:
morphology = data_folder / "morphology" / "morphology_corrected.swc"
probe_file = data_folder / "extracellular" / "probe.json"
params_release = "experimental_model/parameters_release.json"

In [ ]:
cell = mf.create_experimental_model(morphology_file=morphology,
                                    cell_model_folder=model_folder,
                                    release=True,
                                    v_init=-70)

In [ ]:
# define a probe (this can also be defined externally)
probe = mf.define_electrode(probe_file=probe_file)

# define a test protocol
protocols = mf.define_test_step_protocol(step_amplitude=0.5, tot_duration=500, step_duration=400, probe=probe)

# instantiate simulator
sim = ephys.simulators.LFPySimulator(cell, cvode_active=True, electrode=probe, mechs_folders=model_folder)

In [ ]:
t_start = time.time()
responses = protocols["TestStep"].run(cell, param_values={}, sim=sim)
print(f"Elapsed time: {time.time() - t_start}")

In [ ]:
mf.plot_responses(responses)

### Check morphology and probe

In [ ]:
ax = mf.plot_probe(probe, interactive=True)

In [ ]:
ax = mf.plot_cell(cell, sim, ax=ax, color="k", detailed=True)

## Check channel distributions

In [ ]:
cell.freeze({})
cell.instantiate(sim)

In [ ]:
lfpy_cell = cell.LFPyCell
for sec in lfpy_cell.allseclist:
    if "ais" in sec.name():
        break
ais = sec

In [ ]:
plt.figure()
ais_dist = np.linspace(0, ais.L, ais.nseg)
plt.plot(ais_dist, ais.psection()["density_mechs"]["Na16Hu2009"]["gNa16bar"], label="gNa16bar_Na16Hu2009")
plt.plot(ais_dist, ais.psection()["density_mechs"]["Na12Hu2009"]["gNa12bar"], label="gNa12bar_Na12Hu2009")
plt.plot(ais_dist, ais.psection()["density_mechs"]["KdShu2007"]["gkbar"], label="gkbar_KdShu2007")

In [ ]:
dend_list = []
for sec in lfpy_cell.allseclist:
    if "dend" in sec.name():
        dend_list.append(sec)

In [ ]:
plt.figure()
for dend in dend_list:
    start_ = np.array([dend.x3d(0), dend.y3d(0), dend.x3d(0)])
    if dend.nseg > 1:
        start_dist = np.linalg.norm(start_)
        end_dist = start_dist + dend.L
        linear_dist_to_soma = np.linspace(start_dist, end_dist, dend.nseg)        
    else:
        linear_dist_to_soma = [np.linalg.norm(start_) + dend.L // 2]

    plt.plot(linear_dist_to_soma, dend.psection()["density_mechs"]["NaTg"]["gNaTgbar"], "b*")

In [ ]:
cell.unfreeze({})
cell.destroy(sim)